# DW_pandas_Task9

## 20210107

DataWhale_pandas组队学习第九章，分类数据，坚持坚持！！！

### cat对象

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight'])
s = df.Grade.astype('category')
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Junior', 'Senior', 'Sophomore']

在pandas中提供了category类型，使用户能够处理分类类型的变量，将一个普通序列转换成分类变量可以使用astype方法。

In [3]:
s.cat

In [4]:
s.cat.categories   #为类别的本身，它以Index类型存储

Index(['Freshman', 'Junior', 'Senior', 'Sophomore'], dtype='object')

In [5]:
s.cat.ordered   # 为是否有序

False

In [6]:
s.cat.codes.head()   # 每一个序列的类别会被赋予唯一的整数编号，它们的编号取决于cat.categories中的顺序，该属性可以通过codes访问

0    0
1    0
2    2
3    3
4    3
dtype: int8

通过cat对象的categories属性能够完成对类别的查询，那么应该如何进行“增改查删”的其他三个操作呢？

In [7]:
s = s.cat.add_categories('Graduate') # 增加一个毕业生类别
s.cat.categories

Index(['Freshman', 'Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

In [8]:
s = s.cat.remove_categories('Freshman')
s.cat.categories

Index(['Junior', 'Senior', 'Sophomore', 'Graduate'], dtype='object')

In [9]:
s.head()   # 删除某一个类别可以使用remove_categories，同时所有原来序列中的该类会被设置为缺失。

0          NaN
1          NaN
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Junior', 'Senior', 'Sophomore', 'Graduate']

In [10]:
s = s.cat.remove_unused_categories() # 移除了未出现的博士生类别
s.cat.categories   #想要删除未出现在序列中的类别，可以使用remove_unused_categories来实现

Index(['Junior', 'Senior', 'Sophomore'], dtype='object')

In [11]:
s = s.cat.rename_categories({'Sophomore':'本科二年级学生'})
s.head()   # 对类别进行重命名

0        NaN
1        NaN
2     Senior
3    本科二年级学生
4    本科二年级学生
Name: Grade, dtype: category
Categories (3, object): ['Junior', 'Senior', '本科二年级学生']

分类数据具体应该是对数据的量化转变的应用吧。

### 有序分类

In [12]:
s = df.Grade.astype('category')
s = s.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
s.head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman' < 'Sophomore' < 'Junior' < 'Senior']

In [13]:
s.cat.as_unordered().head()

0     Freshman
1     Freshman
2       Senior
3    Sophomore
4    Sophomore
Name: Grade, dtype: category
Categories (4, object): ['Freshman', 'Sophomore', 'Junior', 'Senior']

有序类别和无序类别可以通过as_unordered和reorder_categories互相转化，需要注意的是后者传入的参数必须是由当前序列的无需类别构成的列表，不能够增加新的类别，也不能缺少原来的类别，并且必须指定参数ordered=True，否则方法无效。

如果不想指定ordered=True参数，那么可以先用s.cat.as_ordered()转化为有序类别，再利用reorder_categories进行具体的相对大小调整。

In [14]:
df.Grade = df.Grade.astype('category')
df.Grade = df.Grade.cat.reorder_categories(['Freshman', 'Sophomore', 'Junior', 'Senior'],ordered=True)
df.sort_values('Grade').head() # 值排序

,Grade,Name,Gender,Height,Weight
0,Freshman,Gaopeng Yang,Female,158.9,46.0
105,Freshman,Qiang Shi,Female,164.5,52.0
96,Freshman,Changmei Feng,Female,163.8,56.0
88,Freshman,Xiaopeng Han,Female,164.1,53.0
81,Freshman,Yanli Zhang,Female,165.1,52.0


In [15]:
df.set_index('Grade').sort_index().head() # 索引排序


,Name,Gender,Height,Weight
Grade,,,,
Freshman,Gaopeng Yang,Female,158.9,46.0
Freshman,Qiang Shi,Female,164.5,52.0
Freshman,Changmei Feng,Female,163.8,56.0
Freshman,Xiaopeng Han,Female,164.1,53.0
Freshman,Yanli Zhang,Female,165.1,52.0


把列的类型修改为category后，再赋予相应的大小关系，就能正常地使用sort_index和sort_values。

把无序变为有序排列，可以指定排序关系，可调整性比较大。

In [16]:
res1 = df.Grade == 'Sophomore'
res1.head()

0    False
1    False
2    False
3     True
4     True
Name: Grade, dtype: bool

In [17]:
res2 = df.Grade == ['PhD']*df.shape[0]
res2.head()

0    False
1    False
2    False
3    False
4    False
Name: Grade, dtype: bool

In [18]:
res3 = df.Grade <= 'Sophomore'
res3.head()

0     True
1     True
2    False
3     True
4     True
Name: Grade, dtype: bool

In [19]:
res4 = df.Grade <= df.Grade.sample(frac=1).reset_index(drop=True) # 打乱后比较
res4.head()

0     True
1     True
2     True
3    False
4     True
Name: Grade, dtype: bool

由于序的建立，因此就可以进行比较操作。分类变量的比较操作分为两类，第一种是==或!=关系的比较，比较的对象可以是标量或者同长度的Series（或list），第二种是>,>=,<,<=四类大小关系的比较，比较的对象和第一种类似，但是所有参与比较的元素必须属于原序列的categories，同时要和原序列具有相同的索引。

这个又要结合前课知识，有点蒙，回头去看看。

### 区间类别

区间是一种特殊的类别，在实际数据分析中，区间序列往往是通过cut和qcut方法进行构造的，这两个函数能够把原序列的数值特征进行装箱，即用区间位置来代替原来的具体数值。

cut的常见用法：

最重要的参数是bin，如果传入整数n，则代表把整个传入数组的按照最大和最小值等间距地分为n段。由于区间默认是左开右闭，需要进行调整把最小值包含进去，在pandas中的解决方案是在值最小的区间左端点再减去0.001*(max-min)，因此如果对序列[1,2]划分为2个箱子时，第一个箱子的范围(0.999,1.5]，第二个箱子的范围是(1.5,2]。如果需要指定左闭右开时，需要把right参数设置为False，相应的区间调整方法是在值最大的区间右端点再加上0.001*(max-min)。

In [20]:
s = pd.Series([1,2])
pd.cut(s, bins=2)

0    (0.999, 1.5]
1      (1.5, 2.0]
dtype: category
Categories (2, interval[float64]): [(0.999, 1.5] < (1.5, 2.0]]

In [21]:
pd.cut(s, bins=2, right=False)

0      [1.0, 1.5)
1    [1.5, 2.001)
dtype: category
Categories (2, interval[float64]): [[1.0, 1.5) < [1.5, 2.001)]

一天时间太赶了，忙中抽空，也只是大致看了一点点，都还没领悟进去😳😳